# Parameter optimization

In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['Pattern', 'Trend']
indicator_list = pattern
indicator_list_higher = pattern

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 2000
load = False

print(f'Timeframe is {higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = pattern
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'Pattern': {
                            'use_vol': [0], 
                            'window_low_bound': [1], 
                            'window_high_bound': [5],
                            'first_candle': np.arange(0.5, 1, 0.05),
                            'second_candle': np.arange(0.5, 1, 0.05),
                            'third_candle': [0.5]
                            },
                'Trend': {
                            'timeperiod': [28],
                            'low_bound': [0]
                            }
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

# stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
# if not stat_list:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
# else:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 1h, trade type is sell
Number of combinations is 100


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [26:20<00:00, 15.81s/it]


# Check local statistics

In [3]:
# candles
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg')
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
9,Pattern_Trend,0,1,5,0.50,0.95,0.5,28,0,254,0.874533,0.321250,-31.868533,81.597500,1.0022,1.2120,1.1845,1.1169,1.0777,0.9611,0.9012,0.9278,0.8935,0.8809,0.8197,0.8165,0.7777,0.7638,0.7614,0.7606,0.7497,0.7924,0.8049,0.7882,0.7658,0.7678,0.7427,0.7198,0.04,0.07,0.10,0.13,0.15,0.17,0.19,0.20,0.22,0.25,0.27,0.29,0.31,0.35,0.38,0.42,0.44,0.46,0.49,0.50,0.52,0.57,0.59,0.60
84,Pattern_Trend,0,1,5,0.90,0.70,0.5,28,0,258,0.876000,0.345833,-31.992000,89.225000,1.4681,1.3262,1.1907,1.1142,1.0290,0.9994,0.9349,0.9050,0.8894,0.8814,0.8222,0.7897,0.7659,0.7540,0.7604,0.7467,0.7351,0.7315,0.7285,0.7066,0.6844,0.6828,0.6926,0.6853,0.04,0.08,0.12,0.14,0.17,0.20,0.22,0.25,0.27,0.29,0.31,0.34,0.37,0.40,0.44,0.46,0.48,0.50,0.46,0.50,0.55,0.56,0.57,0.58
81,Pattern_Trend,0,1,5,0.90,0.55,0.5,28,0,367,0.904579,0.353333,-35.019446,129.673333,1.5203,1.3956,1.1797,1.1067,1.0289,1.0260,0.9693,0.9398,0.9154,0.9067,0.8545,0.8280,0.7940,0.7764,0.7790,0.7702,0.7711,0.7709,0.7683,0.7406,0.7178,0.7099,0.7236,0.7172,0.04,0.08,0.12,0.15,0.18,0.21,0.23,0.25,0.27,0.28,0.31,0.34,0.37,0.40,0.44,0.46,0.48,0.51,0.49,0.53,0.57,0.58,0.59,0.60
82,Pattern_Trend,0,1,5,0.90,0.60,0.5,28,0,335,0.921292,0.356667,-26.367292,119.483333,1.5841,1.4439,1.2344,1.1613,1.0722,1.0624,0.9922,0.9571,0.9340,0.9261,0.8678,0.8357,0.8012,0.7834,0.7851,0.7728,0.7656,0.7627,0.7593,0.7382,0.7174,0.7100,0.7249,0.7192,0.04,0.08,0.11,0.14,0.18,0.20,0.22,0.25,0.27,0.29,0.31,0.34,0.38,0.41,0.44,0.47,0.49,0.53,0.50,0.54,0.58,0.59,0.60,0.60
80,Pattern_Trend,0,1,5,0.90,0.50,0.5,28,0,400,0.875875,0.360000,-49.650000,144.000000,1.4480,1.3372,1.1421,1.0863,1.0083,1.0025,0.9432,0.9149,0.8939,0.8778,0.8272,0.8012,0.7658,0.7489,0.7528,0.7453,0.7458,0.7433,0.7426,0.7176,0.6960,0.6867,0.6996,0.6940,0.04,0.08,0.12,0.15,0.18,0.21,0.23,0.25,0.27,0.29,0.31,0.34,0.38,0.41,0.45,0.47,0.49,0.52,0.50,0.54,0.58,0.60,0.61,0.62
19,Pattern_Trend,0,1,5,0.55,0.95,0.5,28,0,226,0.816517,0.361667,-41.467233,81.736667,0.9625,1.0856,1.1046,1.0694,1.0355,0.9122,0.8526,0.8817,0.8525,0.8422,0.7843,0.7776,0.7367,0.7240,0.7222,0.7230,0.7121,0.7046,0.7181,0.7049,0.6845,0.6880,0.6689,0.6487,0.05,0.10,0.13,0.14,0.17,0.19,0.21,0.23,0.25,0.28,0.30,0.32,0.35,0.38,0.41,0.46,0.48,0.51,0.56,0.58,0.61,0.63,0.66,0.68
8,Pattern_Trend,0,1,5,0.50,0.90,0.5,28,0,389,0.895442,0.362500,-40.673192,141.012500,0.8922,1.2739,1.2225,1.1683,1.0751,0.9893,0.9276,0.9433,0.9151,0.8959,0.8466,0.8368,0.8077,0.7988,0.7862,0.7864,0.7814,0.8090,0.8212,0.8057,0.7893,0.7924,0.7726,0.7533,0.05,0.09,0.13,0.16,0.20,0.23,0.25,0.26,0.28,0.31,0.33,0.35,0.38,0.41,0.43,0.46,0.47,0.48,0.52,0.53,0.54,0.60,0.61,0.63
18,Pattern_Trend,0,1,5,0.55,0.90,0.5,28,0,346,0.860275,0.367500,-48.344850,127.155000,0.8740,1.1786,1.1650,1.1342,1.0396,0.9530,0.8984,0.9155,0.8925,0.8754,0.8259,0.8149,0.7838,0.7812,0.7738,0.7736,0.7657,0.7593,0.7696,0.7553,0.7391,0.7433,0.7267,0.7082,0.06,0.10,0.14,0.17,0.20,0.23,0.25,0.27,0.28,0.31,0.33,0.35,0.38,0.41,0.43,0.45,0.46,0.48,0.53,0.55,0.56,0.61,0.62,0.65
83,Pattern_Trend,0,1,5,0.90,0.65,0.5,28,0,306,0.927846,0.370833,-22.079175,113.475000,1.6339,1.4636,1.2640

In [4]:
# LR timeperiod
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,LinearReg_timeperiod,LinearReg_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
41,Pattern_LinearReg,0,1,6,0.65,0.6,0.5,84,0,682,0.629358,1.794167,-252.777617,1223.621667,0.7948,0.7135,0.6854,0.6669,0.6696,0.6435,0.6436,0.6348,0.6086,0.6146,0.6261,0.6318,0.6466,0.6222,0.6180,0.6092,0.6007,0.5950,0.5875,0.5848,0.5818,0.5776,0.5747,0.5733,1.35,1.31,1.34,1.40,1.51,1.59,1.71,1.78,1.73,1.67,1.73,1.80,1.92,1.90,1.92,1.97,1.96,1.99,2.06,2.06,2.05,2.10,2.07,2.14
42,Pattern_LinearReg,0,1,6,0.65,0.6,0.5,86,0,671,0.638054,1.787500,-242.865654,1199.412500,0.7940,0.7147,0.6876,0.6699,0.6727,0.6491,0.6483,0.6388,0.6215,0.6284,0.6398,0.6451,0.6598,0.6340,0.6292,0.6197,0.6110,0.6050,0.5987,0.5956,0.5921,0.5881,0.5859,0.5843,1.35,1.27,1.30,1.38,1.48,1.57,1.69,1.75,1.70,1.67,1.73,1.79,1.91,1.90,1.92,1.97,1.97,2.01,2.08,2.06,2.06,2.11,2.08,2.15
45,Pattern_LinearReg,0,1,6,0.65,0.6,0.5,92,0,636,0.633808,1.786250,-232.897900,1136.055000,0.7871,0.6991,0.6748,0.6657,0.6654,0.6433,0.6398,0.6283,0.6133,0.6227,0.6277,0.6405,0.6571,0.6308,0.6273,0.6180,0.6094,0.6025,0.5997,0.5976,0.5940,0.5900,0.5895,0.5878,1.33,1.23,1.25,1.41,1.52,1.57,1.68,1.75,1.69,1.67,1.72,1.79,1.90,1.92,1.93,1.97,1.97,2.00,2.06,2.08,2.08,2.12,2.07,2.16
39,Pattern_LinearReg,0,1,6,0.65,0.6,0.5,80,0,705,0.620817,1.777083,-267.324250,1252.843750,0.8043,0.7243,0.6917,0.6681,0.6647,0.6377,0.6364,0.6276,0.6008,0.6052,0.6079,0.6148,0.6300,0.6076,0.6028,0.5952,0.5874,0.5831,0.5762,0.5728,0.5692,0.5658,0.5635,0.5625,1.34,1.31,1.32,1.38,1.48,1.57,1.69,1.76,1.70,1.64,1.70,1.77,1.90,1.89,1.91,1.95,1.95,1.98,2.05,2.04,2.04,2.08,2.06,2.14
43,Pattern_LinearReg,0,1,6,0.65,0.6,0.5,88,0,661,0.633246,1.772500,-242.424504,1171.622500,0.7845,0.7060,0.6805,0.6629,0.6636,0.6447,0.6441,0.6338,0.6164,0.6231,0.6281,0.6401,0.6557,0.6301,0.6251,0.6160,0.6075,0.6011,0.5963,0.5940,0.5906,0.5867,0.5846,0.5824,1.36,1.27,1.29,1.37,1.47,1.56,1.67,1.74,1.68,1.65,1.71,1.78,1.90,1.88,1.89,1.94,1.94,1.97,2.04,2.05,2.05,2.10,2.08,2.15
44,Pattern_LinearReg,0,1,6,0.65,0.6,0.5,90,0,645,0.631583,1.770000,-237.628750,1141.650000,0.7884,0.7024,0.6765,0.6611,0.6619,0.6395,0.6372,0.6270,0.6113,0.6201,0.6248,0.6377,0.6543,0.6288,0.6245,0.6154,0.6071,0.6002,0.5957,0.5940,0.5910,0.5872,0.5868,0.5851,1.38,1.27,1.29,1.40,1.50,1.54,1.65,1.72,1.66,1.63,1.69,1.76,1.87,1.89,1.91,1.94,1.94,1.97,2.05,2.06,2.06,2.10,2.06,2.14
38,Pattern_LinearReg,0,1,6,0.65,0.6,0.5,78,0,717,0.617854,1.766250,-273.998562,1266.401250,0.8102,0.7266,0.6890,0.6640,0.6593,0.6326,0.6318,0.6221,0.5971,0.6011,0.6032,0.6098,0.6247,0.6031,0.5990,0.5917,0.5843,0.5803,0.5738,0.5705,0.5675,0.5640,0.5618,0.5610,1.34,1.30,1.31,1.37,1.47,1.56,1.68,1.75,1.69,1.63,1.69,1.76,1.89,1.88,1.90,1.94,1.94,1.97,2.03,2.03,2.03,2.07,2.05,2.11
40,Pattern_LinearReg,0,1,6,0.65,0.6,0.5,82,0,694,0.620867,1.765417,-263.118533,1225.199167,0.7990,0.7195,0.6895,0.6682,0.6660,0.6391,0.6379,0.6294,0.6021,0.6077,0.6106,0.6163,0.6310,0.6074,0.6026,0.5948,0.5868,0.5820,0.5753,0.5730,0.5703,0.5660,0.5637,0.5626,1.36,1.31,1.34,1.39,1.49,1.57,1.69,1.76,1.71,1.64,1.70,1.77,1.89,1.87,1.89,1.93,1.92,1.95,2.01,2.01,2.01,2.05,2.02,2.09
46,Pattern_LinearReg,0,1,6,0.65,0.6,0.5,94,0,626,0.62

In [2]:
# vol
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,LinearReg_timeperiod,LinearReg_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
2,Pattern_LinearReg,1,1,6,0.65,0.6,0.5,60,0,115,0.752329,2.359583,-28.482146,271.352083,0.8766,0.8547,0.8801,0.8510,0.8436,0.8381,0.8372,0.7986,0.7656,0.7438,0.7209,0.6799,0.7087,0.7045,0.6938,0.6955,0.6932,0.6929,0.6911,0.6824,0.6817,0.7043,0.7131,0.7046,1.74,1.69,1.41,1.44,1.74,1.88,1.93,2.17,2.47,2.36,2.53,2.21,2.39,2.46,2.47,2.72,2.86,2.77,2.64,2.70,2.80,3.09,3.10,3.06
1,Pattern_LinearReg,0,1,6,0.65,0.6,0.5,60,0,831,0.596792,1.729167,-335.066125,1436.937500,0.7929,0.6930,0.6596,0.6330,0.6306,0.6158,0.6155,0.6084,0.5846,0.5880,0.5863,0.5880,0.6025,0.5836,0.5787,0.5716,0.5645,0.5597,0.5538,0.5491,0.5417,0.5393,0.5430,0.5398,1.36,1.31,1.35,1.40,1.46,1.55,1.67,1.75,1.73,1.66,1.73,1.75,1.86,1.81,1.84,1.88,1.85,1.87,1.91,1.90,1.91,1.97,1.96,2.02
0,Pattern_LinearReg,-1,1,6,0.65,0.6,0.5,60,0,550,0.563358,1.591250,-240.152917,875.187500,0.8128,0.6830,0.6199,0.5875,0.5791,0.5692,0.5622,0.5576,0.5401,0.5536,0.5557,0.5618,0.5802,0.5610,0.5561,0.5448,0.5347,0.5262,0.5189,0.5127,0.5047,0.5005,0.5000,0.4983,1.47,1.39,1.42,1.47,1.48,1.59,1.72,1.70,1.64,1.61,1.66,1.66,1.67,1.65,1.65,1.65,1.59,1.58,1.63,1.57,1.56,1.61,1.56,1.66


In [2]:
# high/low bound
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('e_ratio_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,LinearReg_timeperiod,LinearReg_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
25,Pattern_LinearReg,1,1,5,0.65,0.6,0.5,60,0,100,0.801304,2.155000,-19.869583,215.500000,0.8803,0.8638,0.9015,0.8847,0.8872,0.8763,0.8660,0.8237,0.7865,0.7615,0.7371,0.7082,0.7594,0.7626,0.7532,0.7582,0.7654,0.7770,0.7678,0.7597,0.7646,0.7922,0.8037,0.7907,1.54,1.48,1.53,1.53,1.80,1.91,1.92,2.13,2.41,2.24,2.36,1.97,2.07,2.13,2.15,2.41,2.49,2.41,2.30,2.36,2.45,2.71,2.73,2.69
29,Pattern_LinearReg,1,2,5,0.65,0.6,0.5,60,0,96,0.768983,2.104167,-22.177600,202.000000,0.8424,0.8366,0.8658,0.8489,0.8434,0.8311,0.8295,0.7936,0.7568,0.7339,0.7120,0.6857,0.7378,0.7322,0.7218,0.7245,0.7310,0.7436,0.7348,0.7271,0.7313,0.7598,0.7722,0.7598,1.59,1.41,1.39,1.37,1.64,1.76,1.96,2.18,2.46,2.29,2.42,2.07,2.11,2.15,2.16,2.39,2.45,2.34,2.21,2.25,2.32,2.55,2.55,2.48
24,Pattern_LinearReg,1,1,4,0.65,0.6,0.5,60,0,79,0.752937,1.778750,-19.517938,140.521250,0.8673,0.8480,0.8642,0.8292,0.8480,0.8411,0.8694,0.8380,0.7883,0.7570,0.7329,0.6968,0.6880,0.6878,0.6786,0.6833,0.6969,0.7083,0.6967,0.6842,0.6837,0.6928,0.7024,0.6876,1.54,1.43,1.65,1.54,1.78,1.76,1.63,1.75,1.96,1.78,1.82,1.36,1.62,1.66,1.68,1.91,1.97,1.89,1.84,1.88,1.93,2.10,2.10,2.11
26,Pattern_LinearReg,1,1,6,0.65,0.6,0.5,60,0,115,0.752329,2.359583,-28.482146,271.352083,0.8766,0.8547,0.8801,0.8510,0.8436,0.8381,0.8372,0.7986,0.7656,0.7438,0.7209,0.6799,0.7087,0.7045,0.6938,0.6955,0.6932,0.6929,0.6911,0.6824,0.6817,0.7043,0.7131,0.7046,1.74,1.69,1.41,1.44,1.74,1.88,1.93,2.17,2.47,2.36,2.53,2.21,2.39,2.46,2.47,2.72,2.86,2.77,2.64,2.70,2.80,3.09,3.10,3.06
33,Pattern_LinearReg,1,3,5,0.65,0.6,0.5,60,0,65,0.738450,2.276250,-17.000750,147.956250,0.8527,0.7368,0.7879,0.8357,0.8140,0.7957,0.7589,0.7173,0.6854,0.6724,0.6367,0.6087,0.6977,0.7056,0.6923,0.6888,0.7064,0.7283,0.7299,0.7309,0.7505,0.7857,0.8146,0.7899,0.93,0.90,0.83,1.05,1.35,1.74,2.05,2.31,2.73,2.72,3.06,2.80,2.65,2.64,2.63,2.72,2.73,2.58,2.45,2.51,2.57,2.90,2.95,2.83
30,Pattern_LinearReg,1,2,6,0.65,0.6,0.5,60,0,111,0.725154,2.318333,-30.507888,257.335000,0.8446,0.8319,0.8502,0.8212,0.8078,0.8007,0.8065,0.7731,0.7403,0.7203,0.6994,0.6605,0.6899,0.6785,0.6671,0.6669,0.6643,0.6650,0.6635,0.6553,0.6541,0.6772,0.6867,0.6787,1.80,1.64,1.29,1.29,1.59,1.74,1.97,2.22,2.52,2.42,2.58,2.30,2.43,2.49,2.48,2.71,2.83,2.72,2.57,2.60,2.68,2.95,2.94,2.88
28,Pattern_LinearReg,1,2,4,0.65,0.6,0.5,60,0,75,0.710867,1.707917,-21.685000,128.093750,0.8173,0.8134,0.8176,0.7834,0.7921,0.7830,0.8220,0.7988,0.7496,0.7213,0.7006,0.6678,0.6585,0.6474,0.6371,0.6389,0.6519,0.6647,0.6538,0.6420,0.6405,0.6504,0.6613,0.6474,1.60,1.36,1.49,1.36,1.59,1.57,1.67,1.80,2.02,1.83,1.87,1.44,1.65,1.68,1.67,1.88,1.91,1.80,1.72,1.74,1.76,1.88,1.86,1.84
34,Pattern_LinearReg,1,3,6,0.65,0.6,0.5,60,0,79,0.684817,2.576250,-24.899483,203.523750,0.8354,0.7409,0.7756,0.7933,0.7660,0.7569,0.7417,0.7087,0.6802,0.6683,0.6360,0.5929,0.6458,0.6425,0.6288,0.6231,0.6263,0.6297,0.6373,0.6345,0.6420,0.6684,0.6878,0.6735,1.34,1.33,0.78,1.01,1.32,1.71,2.04,2.35,2.77,2.83,3.20,3.05,3.06,3.11,3.08,3.16,3.29,3.11,2.95,3.01,3.07,3.43,3.47,3.36
27,Pattern_LinearReg,1,1,7,0.65,0.6,0.5,60,0,136,0.679908,2.382917,-43.532

# Research parameter influence

In [8]:
param = 'Pattern_second_candle'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Pattern_second_candle,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,0.5,0.854897,-0.304701,-168.318122,-286.847375,68130
1,0.6,0.850164,-0.288188,-150.008786,-258.243681,60370
2,0.7,0.849519,-0.281743,-133.779457,-224.315035,52007
3,0.8,0.839842,-0.299785,-109.302578,-205.237062,41617
4,0.9,0.871600,-0.322160,-72.804460,-170.303528,30776


# Plot result in file

In [11]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = param, 'e_ratio_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig(f"{'_'.join(pattern)}'_'{ttype}_{param}", dpi=fig.dpi)

# Save new config data to config file

In [4]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['Pattern', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                 'Pattern': {
                            'use_vol': [0], 
                            'window_low_bound': [1], 
                            'window_high_bound': [4],
                            'first_candle': [0.8],
                            'second_candle': [0.5],
                            'third_candle': [0.5]
                            },
                'Trend': {
                            'timeperiod': [28], 
                            'low_bound': [0]
                            }
             }
 
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)